# Registar um modelo

Este notebook é similar ao 02. Neste notebook vamos criar uma nova versão do mesmo modelo e assim perceber e experienciar um model registry.

In [1]:
import mlflow

In [2]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [3]:
ROOT_PATH = '../../data/'
SEED = 1
TARGET_COL = "Outcome"

## Definir a diretoria onde as experiências são guardadas

In [4]:
from pathlib import Path

uri = "../../mlruns/local"

Path(uri).mkdir(parents=True, exist_ok=True)

mlflow.set_tracking_uri(uri)

## Fazer set da experiência "Diabetes Prediction Experiment"

In [5]:
mlflow.set_experiment("Diabetes Prediction Experiment")

<Experiment: artifact_location='/home/mgl/rumos/notebooks/mlflow/../../mlruns/local/761797969699593739', creation_time=1731101986464, experiment_id='761797969699593739', last_update_time=1731101986464, lifecycle_stage='active', name='Diabetes Prediction Experiment', tags={}>

## Criar os datasets

In [6]:
train_path = ROOT_PATH + 'diabetes_train.csv'
test_path = ROOT_PATH + 'diabetes_test.csv'

train_set = pd.read_csv(train_path)
test_set = pd.read_csv(test_path)

X_train = train_set.drop([TARGET_COL], axis = 1)
y_train = train_set[TARGET_COL]

X_test = test_set.drop([TARGET_COL], axis = 1)
y_test = test_set[TARGET_COL]

X_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,3,173,78,39,185,33.8,0.970,31
1,6,134,80,37,370,46.2,0.238,46
2,5,104,74,0,0,28.8,0.153,48
3,1,139,46,19,83,28.7,0.654,22
4,5,137,108,0,0,48.8,0.227,37


## Criar uma run

In [7]:
run = mlflow.start_run(run_name="Linear Regression Run - C0.1")
RUN_ID = run.info.run_uuid
RUN_ID

'6f521b03b7bd40f68d1800b091063363'

## Guardar datasets, modelos, artefactos, métricas e parametros da run

In [8]:
# guardarmos o dataset de treino e de teste associado à run
train_dataset = mlflow.data.from_pandas(train_set, source=train_path, targets=TARGET_COL, name="Diabetes Train Dataset")
test_dataset = mlflow.data.from_pandas(test_set, source=test_path, targets=TARGET_COL, name="Diabetes Test Dataset")
mlflow.log_input(train_dataset, context="train")
mlflow.log_input(test_dataset, context="test")

# Guardamos a seed utilizado como parametro
mlflow.log_param("seed", SEED)

/home/mgl/miniconda3/envs/OML/lib/python3.11/site-packages/mlflow/data/dataset_source_registry.py:149: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for '../../data/diabetes_train.csv'. Exception: 
  return _dataset_source_registry.resolve(
/home/mgl/miniconda3/envs/OML/lib/python3.11/site-packages/mlflow/data/dataset_source_registry.py:149: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
/home/mgl/miniconda3/envs/OML/lib/python3.11/site-packages/mlflow/data/dataset_source_registry.py:149: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for '../../data/diabetes_test.csv'. Exception: 
  return _dataset_source_registry.resolve(
/home/mgl/miniconda3/envs/OML/lib/python3.11/site-packages/mlflow/data/da

1

In [9]:
logistic_reg = LogisticRegression(random_state = SEED, C=0.1)
logistic_reg.fit(X_train, y_train)
mlflow.sklearn.log_model(logistic_reg, artifact_path="logistic_reg", registered_model_name="logistic_reg")

logistic_reg

/home/mgl/miniconda3/envs/OML/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
2024/11/09 10:04:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'logistic_reg' already exists. Creating a new version of this model...
Created version '2' of model 'logistic_reg'.


LogisticRegression(C=0.1, random_state=1)

In [10]:
params=logistic_reg.get_params()
mlflow.log_params(params)
params

{'C': 0.1,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'deprecated',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': 1,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [11]:
y_preds = logistic_reg.predict(X_test)
acc = accuracy_score(y_test, y_preds)
mlflow.log_metric("accuracy", acc)
acc

0.7105263157894737

## Terminar a run

In [12]:
mlflow.end_run()

## Consultar uma run já concluida

In [13]:
run = mlflow.get_run(RUN_ID)

In [14]:
run.data

<RunData: metrics={'accuracy': 0.7105263157894737}, params={'C': '0.1',
 'class_weight': 'None',
 'dual': 'False',
 'fit_intercept': 'True',
 'intercept_scaling': '1',
 'l1_ratio': 'None',
 'max_iter': '100',
 'multi_class': 'deprecated',
 'n_jobs': 'None',
 'penalty': 'l2',
 'random_state': '1',
 'seed': '1',
 'solver': 'lbfgs',
 'tol': '0.0001',
 'verbose': '0',
 'warm_start': 'False'}, tags={'mlflow.log-model.history': '[{"run_id": "6f521b03b7bd40f68d1800b091063363", '
                             '"artifact_path": "logistic_reg", '
                             '"utc_time_created": "2024-11-09 '
                             '10:04:05.286652", "model_uuid": '
                             '"b50b2f10907b465d8cae807c7cab5580", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"predict_fn": "predict", "loader_module": '
                             '"mlflow.sklearn", "python_version": "3.11.10", '
                  

## Opcional - Ver a experiência na UI do mlflow

A UI do mlflow permite ver de forma visual todas as experiências criadas e permite por exemplo, comparar, filtar e ordenar, as runs dentro de uma experiência de forma visual.

Para correr a UI do mflow é necessário executar, na raiz deste projeto (pasta rumos) e tendo activo o ambiente utilizado neste projeto, o comando:

`mlflow ui --backend-store-uri ./mlruns/local`

**Nota:** O comando em cima irá iniciar a UI de mlflow na porta 5000. Caso queiram mudar esta porta devem acrescentar `--port <PORT>` ao comando (em que <PORT> deve ser substituido pela porta desejada). 

O comando acima não irá funcionar caso tenham tido alguns problemas no Windows com a instalação do mlflow, mas como o título desta secção indica este passo é apenas opcional e não irá ser avaliado.

Após executarem este comando, vão poder ver a UI do mlflow no vosso browser acedendo a 

`http://127.0.0.1:5000`

(se tiverem alterado a porta em que o mlflow UI é iniciado então devem de alterar também aqui o 5000 por essa porta)

Na tab de `Experiments` podem explorar as experiências e runs que criaram.